## THEx Model

The following section illustrates how to call and run a model in THEx infrastructure. There are three models: the binary classifiers (BinaryModel), the One-Vs-All classifier (that aggregates the binary results, the OvAModel), and the KDE multiclass classifier which creates a unique KDE for each class and normalizes over those likelihoods (MultiModel). 

These are the following parameters the models handle:
- __cols__ [default=None] : List of column/feature names to use ; the default is all numeric columns
- __col_matches__ [default=None]: An alternative to passing in column names. Here a list of strings may be passed on, and any column containing one of these strings will be used. If both cols and col_matches are set, only col_matches is used 
- __num_runs__ [default=None]: The number of trials to run and average results over. For each trial, 80% of data will be randomly selected for training, and 20% for testing. 
- __folds__ [default=None] : The number of folds to run over, in k-fold cross-validation. If both num_runs and folds are passed in, num_runs will be used.
- __transform_features__ [default=True]: Derives colors from adjacent magnitudes, using dictionary ORDERED_MAGS in thex_data.data_consts.py
- __min_class_size__ [default=9]: Each class must contain at least this number of samples for it to be used. 
- __max_class_size__ [default=None]: Classes with more than this number of samples will be randomly sampled down to this number
- __pca__ [default=None]: Number of components to reduce down to using PCA, by default there is no PCA
- __class_labels__ [default=None]: List of classes to limit analysis to. List of all classes is in thex_data.data_consts, ORDERED_CLASSES
- __data__ [default=None]: Optional parameter for testing particular sets of data. By default, we collect the data from the file in thex_data.data_consts DATA_PATH file, but this parameter may be used to pass in particular datasets. It must be a list of the training and testing Pandas DataFrames: [train_df, test_df] 
- __nb__ [default=False]: Boolean on applying Naive Bayes. If True, a unique KDE is created for each dimension. If False, we use multivariate KDE. 
- __priors__ [default=None]: Prior probabilities to use. If None, no priors are used (uniform priors assumed). Otherwise, a list may be passed in with the prior probability for each class, in the same order as classes are listed in class_labels. 

In [ ]:
%matplotlib inline  
from models.binary_model.binary_model import BinaryModel
from models.ind_model.ind_model import OvAModel
from models.multi_model.multi_model import MultiModel


mags = ["g_mag",  "r_mag", "i_mag", "z_mag", "y_mag",
        "W1_mag", "W2_mag",
        "J_mag", "K_mag", "H_mag"]

# mags = ["g_mag",  "r_mag", "i_mag", "z_mag", "y_mag", "u_mag",
#         "W1_mag", "W2_mag", "W3_mag", "W4_mag",
#         "J_mag", "K_mag", "H_mag",
#         "NUV_mag", "FUV_mag"]

model = MultiModel(
       cols = mags,
       folds = 10,  
#        class_labels = ['Unspecified Ia', 'Unspecified II', 'TDE', 'Ia-91bg'],
       min_class_size = 40, 
#        max_class_size =  1000,
       transform_features = False
       )
model.visualize_data()
model.run_model() 

## Load previous runs

In [ ]:
# Rerun performance visualizations on saved output of model. 

import pickle

pickle_dir = "../../../experiments/paper_set/multi/Multiclass_Classifier1/"

with open(pickle_dir + 'results.pickle', 'rb') as handle:
    results = pickle.load(handle)

with open(pickle_dir + 'y.pickle', 'rb') as handle:
    y = pickle.load(handle)    

model.y = y
model.results = results
# model.visualize_performance()

## Plot example outputs
For given model

In [ ]:
import random
import numpy as np
import os
import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from mainmodel.helper_compute import *
from thex_data.data_consts import *
import utilities.utilities as thex_utils


def plot_example_output(model, row, i=None, priors=None):
    """
    Plots example output for a set of probabilities for a particular host-galaxy
    :param row: Numpy array of probabilities in order of self.class_labels and then TARGET_LABEL
    :param i: Index of sample
    :param priors: Boolean if using priors, for saving
    """
    labels = row[len(row) - 1]
    true_class_index = None
    for class_index, class_name in enumerate(model.class_labels):
        if class_name in thex_utils.convert_str_to_list(labels):
            true_class_index = class_index

    f, ax = plt.subplots(figsize=(5, 3), dpi=220)

    ACC = "#b3e0ff"  # actual class color, light blue
    DCC = "#005c99"  # default class color, dark blue
    if priors:
        ACC = "#00ffbf"  # actual class color, light green
        DCC = "#00664d"  # default class color, dark green 
        
    colors = [DCC] * len(model.class_labels)
    colors[true_class_index] = ACC
    probabilities = row[0:len(row) - 1] 
    x_indices = np.linspace(0,
                            len(model.class_labels) * 0.4,
                            len(model.class_labels))
    ax.bar(x=x_indices, height=probabilities,  width=0.4, color=colors, bottom=-0.01)
    print("\n Probs for this sample")
    print(probabilities)

    plt.xlabel('Class', fontsize=LAB_S)
    pretty_class_names = clean_class_names(model.class_labels)
    plt.xticks(x_indices, pretty_class_names, fontsize=TICK_S)
    
    ax.set_ylim([0, 1])
    yticks = np.arange(0,1.2,.2) 
    plt.yticks(yticks, [str(int(i*100)) + "%" for i in yticks], fontsize=TICK_S)
    plt.ylabel('Probability Assigned', fontsize=LAB_S)
    
    f.set_title(i)
    plt.show() 

def get_sample_name(model, sample):
#     sample = model.X.iloc[2] 
    features = list(model.X)
    match_str = None
    for feature in features:
        
        v = full_data_set[full_data_set[feature] == sample[feature]] 
        n= v['name'].values[0]
        if match_str is None:
            match_str = n
        else:
            if match_str != n:
                raise ValueError("More than 1")
    return match_str

def plot_cor_examples(model_with, model_wo, sample_index=None):
    """
    Plot corresponding example outputs, with and without priors
    :param row: Numpy array of probabilities in order of self.class_labels and then TARGET_LABEL
    :param i: Index of sample
    :param priors: Boolean if using priors, for saving
    """ 
    
    
    labels = model_with.y.iloc[sample_index]['transient_type']
    
    true_class_index = None
    for class_index, class_name in enumerate(model_with.class_labels):
        if class_name in thex_utils.convert_str_to_list(labels):
            true_class_index = class_index 
    f, ax = plt.subplots(nrows=1,
                         ncols=2,
                         sharex=True, sharey=True,
                         figsize=(6, 2),
                         dpi=200) 
    
    x_indices = np.linspace(0, len(model_with.class_labels) * 0.4,  len(model_with.class_labels)) 
    X_example = model_with.X.iloc[sample_index]
    model_with_ps = list(model_with.get_class_probabilities(X_example).values())
    model_without_ps = list(model_wo.get_class_probabilities(X_example).values())
    
    plt.rcParams['xtick.labelsize'] = 8.5 
    pretty_class_names = clean_class_names(model_wo.class_labels)
#     plt.xticks(x_indices, pretty_class_names, rotation=-20)
    
    ACC = "#b3e0ff"  # actual class color, light blue
    DCC = "#005c99"  # default class color, dark blue
    colors = [DCC] * len(model_with.class_labels)
    colors[true_class_index] = ACC 
    ax[0].bar(x=x_indices, height=model_without_ps,  width=0.4, color=colors, bottom=0)
    ax[0].set_title("Without Priors", fontsize=10)
    
    ACC = "#00ffbf"  # actual class color, light green
    DCC = "#00664d"  # default class color, dark green 
    colors = [DCC] * len(model_with.class_labels)
    colors[true_class_index] = ACC 
    ax[1].bar(x=x_indices, height=model_with_ps,  width=0.4, color=colors, bottom=0)
    ax[1].set_title("With Priors", fontsize=10)
    ax[0].set_ylim([0, 1])
    ax[1].set_ylim([0, 1])
    yticks = np.arange(0,1.2,.2) 
    plt.yticks(yticks, [str(int(i*100)) + "%" for i in yticks])
    ax[0].set_ylabel('Probability Assigned')
    
    
    n = get_sample_name(model_with, model_with.X.iloc[sample_index])
    plt.figtext(0.5, 1, n, ha='center', va='center') 
    plt.tight_layout()
    plt.xticks(x_indices, pretty_class_names)
#     plt.gcf().subplots_adjust(top=1.5)
    plt.savefig("../output/custom_figures/" + str(sample_index) + ".pdf",bbox_inches='tight')
    plt.show()

def plot_sample(model, priors, sample_index):
    X_example = model.X.iloc[sample_index]
    y_example = model.y.iloc[sample_index]
    # Get probs and convert from map to list 
    ps = list(model.get_class_probabilities(X_example).values())
    r = np.hstack((ps, y_example.values)) # combine w/ label
    plot_example_output(model, r, sample_index, priors) 

def plot_new_samples(model, num_samples, indices, priors):
    """
    Randomly sample from this list and plot examples
    :param num_samples: The number of samples to randomly sample
    :param indices: all indices for a particular class to sample from
    :param priors: Boolean
    """
    rand_indices = np.random.choice(indices, num_samples, replace=False)
    for sample_index in rand_indices:
        plot_sample(model, priors, sample_index)
    return rand_indices


In [ ]:
model.results[0][10]

In [ ]:
for index, row in enumerate(model.results[0]):
    if 'GRB' in row[len(model.class_labels)]:
        print(index)

In [ ]:
a=[2,3,3,33]
a.reverse()
b

In [ ]:
import numpy as np
b=np.array([2,3,3,33])
b.tolist()

In [ ]:
do_index = 733
do_index = 800
while do_index< 880:
    
    row = model.results[0][do_index]
    # 1015
    labels = row[len(row) - 1]
    true_class_index = None
    for class_index, class_name in enumerate(model.class_labels):
        if class_name in thex_utils.convert_str_to_list(labels):
            true_class_index = class_index

    f, ax = plt.subplots(figsize=(2, 4), dpi=400)

    ACC = "#005c99"# actual class color, light blue
    DCC = "#b3e0ff" # default class color, dark blue
    # if priors:
    #     ACC = "#00ffbf"  # actual class color, light green
    #     DCC = "#00664d"  # default class color, dark green 

    colors = [DCC] * len(model.class_labels)
    colors[true_class_index] = ACC
    probabilities = row[0:len(row) - 1] 
    bar_size = 0.1
    x_indices = np.linspace(0,
                            len(model.class_labels) * bar_size,
                            len(model.class_labels))

    # x_list = x_indices.tolist()
    # x_list.reverse()
    # pretty_class_names.reverse()
    # probabilities = probabilities.tolist()
    # probabilities.reverse()
    # print(x_list)
    # print(probabilities)
    ax.barh(y=x_indices, width=probabilities,  height=bar_size, color=colors)#, bottom=-0.01)

    # plt.xlabel('Class', fontsize=LAB_S)
    pretty_class_names = clean_class_names(model.class_labels)
    plt.yticks(x_indices, pretty_class_names, fontsize=8)

    ax.set_xlim([0, 1])
    yticks = np.arange(0,1.2,.2) 
    plt.xticks(yticks, [str(int(i*100)) + "%" for i in yticks], fontsize=9, rotation=-90)
    plt.xlabel('Probability', fontsize=10)

    # f.set_title(i)
    plt.gcf().subplots_adjust(left=0.42)
    plt.gcf().subplots_adjust(bottom=0.15)
    plt.savefig("../output/custom_figures/" + str(do_index)+".pdf")
    plt.show() 
    
    do_index+=1


## Plot example outputs
For model with vs. without priors

In [ ]:
import warnings
warnings.filterwarnings('ignore')
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
%matplotlib inline  
from models.binary_model.binary_model import BinaryModel
from models.ind_model.ind_model import IndModel
from models.multi_model.multi_model import MultiModel
 
import pickle



# Changes from other approach - we do not scale data ; otherwise testing new points is tricky. 

multi_w = MultiModel(folds = 2,
                   min_class_size = 40,  
                   class_labels = ['Unspecified Ia', 'Unspecified II', 'Ia-91bg', 'TDE'],
                   priors = [0.65, 0.36, 0.01, 0.005],
                   transform_features = False,
                   cols = mags)  
# multi_w = load_prev_exp(expnum="106/Multiclass_Classifier13",model=model)
multi_w.run_model()


multi_wo = MultiModel(folds = 2,
                   min_class_size = 40,  
                   class_labels = ['Unspecified Ia', 'Unspecified II', 'Ia-91bg', 'TDE'], 
                   transform_features = False,
                   cols = mags)  
# multi_wo = load_prev_exp(expnum="107/Multiclass_Classifier15",model=model)
multi_wo.run_model()



from thex_data.data_init import collect_data
full_data_set=collect_data()



In [ ]:
# Plot examples for each class, first WITHOUT prior

# model = multi_wo
class_indices_map = {}
for cur_class in multi_wo.class_labels:
    indices = []
    print("\nSampling class: " + str(cur_class))
    for index, row in multi_wo.y.iterrows():
        if cur_class in row['transient_type']:
            indices.append(index)
    rand_indices = np.random.choice(indices, 3, replace=False)
    for index in rand_indices:
        plot_cor_examples(multi_w, multi_wo, sample_index=index) 